In [1]:
import tensorflow as tf
tf.enable_eager_execution()

!sudo rm -f /etc/boto.cfg
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers

import saving_train
import loading_ds_train

import loading_ds_test
import saving_test

In [2]:
## Must be openslide version 3.4.1
from openslide import open_slide, __library_version__ as openslide_version
print(openslide_version)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

3.4.1


In [3]:
## Optional: remove sliced images from directory if necessary
!rm -r 'data'

In [5]:
training_image_path_list = ['tumor_094.tif']

In [ ]:
saving_train.train_part_1(training_image_path_list,num_pixels = 64, num_level = 3)

tumor_094.tif tumor_094_mask.tif


In [ ]:
ds_train, steps_per_epoch_train = loading_ds_train.train_part_2(training_image_path_list)

In [ ]:
## Create sequential model with Imagenet base
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(128, 128, 3))

conv_base.trainable = False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
## Compile and train model
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss=tf.keras.losses.sparse_categorical_crossentropy, 
              metrics = ['acc'])
model.summary()

history = model.fit(ds_train, epochs=2, steps_per_epoch=steps_per_epoch_train)

In [ ]:
## Plot results
print(history.history)

acc = history.history['acc']
loss = history.history['loss']
# val_acc = history.history['val_loss']
# val_loss = history.history['val_acc']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b-', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b-', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# image from train set
image_path = 'tumor_091.tif'
image_depth, image_width, tumor_mask, tissue_regions, slide_image_test = saving_test.test_part_1(image_path, num_pixels=64, num_level=2)
loading_ds_test.test_part_2(image_path,model,tissue_regions, slide_image_test, tumor_mask, image_depth, image_width)

In [13]:
# new image from test set
image_path = 'tumor_101.tif'
image_depth, image_width, tumor_mask, tissue_regions, slide_image_test = saving_test.test_part_1(image_path, num_pixels=64, num_level=2)


tumor_101.tif tumor_101_mask.tif
0 0


MemoryError: Integer overflow in ysize

In [ ]:
loading_ds_test.test_part_2(image_path,model,tissue_regions, slide_image_test, tumor_mask, image_depth, image_width)
